In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molsysmt as msm
from molsysmt import puw

# Adding solvent

In [3]:
molsys = 'GRKFRRKFKK' # Cationic peptide
molsys = msm.build_peptide([molsys, {'forcefield':'AMBER14'}], to_form='molsysmt.MolSys')
molsys = msm.add_terminal_capping(molsys, N_terminal='ACE', C_terminal='NME')

In [4]:
msm.physchem.charge([molsys, {'forcefield':'AMBER14'}], target='system')

7.0 <Unit('elementary_charge')>

In [5]:
msm.is_solvated(molsys)

False

In [6]:
msm.info(molsys)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_peptides,n_frames
molsysmt.MolSys,219,12,1,1,1,1,1,1


In [7]:
molsys_cub = msm.solvate([molsys, {'forcefield':'AMBER14', 'water_model':'TIP3P'}],
                          box_geometry='cubic', clearance=14.0*puw.unit('angstroms'),
                          to_form='molsysmt.MolSys', engine="OpenMM", verbose=False)

In [8]:
msm.is_solvated(molsys_cub)

True

In [9]:
msm.info(molsys_cub)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_waters,n_ions,n_peptides,n_frames
molsysmt.MolSys,28489,9440,9429,3,9429,3,9421,7,1,1


In [10]:
msm.info(molsys_cub, target='entity')

index,name,type,n atoms,n groups,n components,n chains,n molecules
0,Peptide_0,peptide,219,12,1,1,1
1,water,water,28263,9421,9421,1,9421
2,CL,ion,7,7,7,1,7


In [11]:
msm.physchem.charge([molsys_cub, {'forcefield':'AMBER14', 'water_model':'TIP3P'}], target='system')

-0.0 <Unit('elementary_charge')>

In [12]:
box, box_angles = msm.get(molsys_cub, target='system', box=True, box_angles=True)

In [13]:
box

Magnitude,[[[6.628617744380609 0.0 0.0] [0.0 6.628617744380609 0.0] [0.0 0.0 6.628617744380609]]]
Units,nanometer


In [14]:
box_angles

Magnitude,[[90.000001 90.000001 90.000001]]
Units,degree


In [15]:
molsys_cub = msm.wrap_to_pbc(molsys_cub, center_of_selection='molecule_type=="peptide"')

In [16]:
msm.view(molsys_cub, standardize=True, water_as_surface=True)

NGLWidget()

## Adding ions 

## PBC box geometry

In [22]:
molsys_oct = msm.solvate(molsys, box_geometry='truncated_octahedral',
                         clearance=14.0*puw.unit('angstroms'), engine='PDBFixer')

In [23]:
msm.info(molsys_oct)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_waters,n_ions,n_peptides,n_frames
molsysmt.MolSys,10759,3530,3519,3,3519,3,3511,7,1,1


In [24]:
molsys_oct = msm.wrap_to_pbc(molsys_oct, center_of_selection='molecule_type=="peptide"')

In [25]:
msm.view(molsys_oct, standardize=True, water_as_surface=True)

NGLWidget()

In [26]:
molsys_oct = msm.wrap_to_mic(molsys_oct, center_of_selection='molecule_type=="peptide"')

In [27]:
msm.view(molsys_oct, standardize=True, water_as_surface=True)

NGLWidget()

In [ ]:
molecular_system_solvated = msm.solvate([msm.add_missing_hydrogens(molecular_system),
                                         {'forcefield':'AMBER14', 'water_model':'TIP3P'}],
                                        box_geometry='truncated_octahedral',
                                        clearance=14.0*puw.unit('angstroms'),
                                        engine='OpenMM', verbose=False)

In [ ]:
msm.info(molecular_system_solvated)

In [ ]:
molecular_system_solvated = msm.solvate([msm.add_missing_hydrogens(molecular_system),
                                         {'forcefield':'AMBER14', 'water_model':'TIP3P'}],
                                        box_geometry='rhombic_dodecahedron',
                                        clearance=14.0*puw.unit('angstroms'),
                                        engine='OpenMM', verbose=False)

In [ ]:
msm.info(molecular_system_solvated)